<h1>Scraping Fidelity.com</h1>
In this assignment, you will scrape data from fidelity.com. The goal of the exercise is to get the latest sector performance data from the US markets, and to get the total market capitalization for each sector. 

The end result is to write a function: <i>get_us_sector_performance()</i> that will return a list of tuples. Each tuple should correspond to a sector and should contain the following data:
<li>the sector name
<li>the amount the sector has moved
<li>the market capitalization of the sector
<li>the market weight of the sector
<li>a link to the fidelity page for that sector

<p>
The data should be sorted by decreasing order of market weight. I.e., the sector with the highest weight should be in the first tuple, etc.

<h3>Process</h3>
<li>Get a list of sectors and the links to the sector detail pages from the url (see function)
<li>Loop through the list and call the function <i>get_sector_change_and_market_cap(sector_page_link)</i> for each sector
<li>Accumulate the name, the change, the capitalization, the weight and the link for each sector in output_list (see function)
<li>Sort the list by market weight

<b>Notes:</b>
<li>Note that the market weight is a string with a % sign at the back. You will need to get rid of the % and convert the string into a float before you can sort it
<li>Your starting data is the url listed below. You need to extract all data, including links to the sector pages, from the page at this url
<li>To sort a list of tuples by an arbitrary element, use the example at the bottom of this notebook

In [1]:
import requests
! pip install bs4
from bs4 import BeautifulSoup
! pip install lxml

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=14faa9b5a4f677740ff5cf4e062dacaaa0a6d3dc18470de878312cd7137dfcbb
  Stored in directory: /Users/zhenqiangzeng/Library/Caches/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [2]:
def get_us_sector_performance():
    output_list = list()
    url = "https://eresearch.fidelity.com/eresearch/goto/markets_sectors/landing.jhtml"
    response = requests.get(url)
    results_page = BeautifulSoup(response.content,'html.parser')
    sectors = results_page.find_all('a',class_ = "heading1")
    for sector in sectors:
        sector_name = sector.get_text()
        link = sector.get('href')
        sector_page_link = 'https://eresearch.fidelity.com' + link[:-6] + '&sect' + link[-5:]
        sector_change,sector_market_cap,sector_market_weight = get_sector_change_and_market_cap(sector_page_link)
        output_list.append((sector_name,sector_change,sector_market_cap,sector_market_weight,sector_page_link))
    output_list.sort(key = lambda x:x[3],reverse = True)
    return output_list
    

In [3]:
def get_sector_change_and_market_cap(sector_page_link):
    response = requests.get(sector_page_link)
    sector_result_page = BeautifulSoup(response.content,'html.parser')
    results = sector_result_page.find_all('td')
    result_temp = results[0].find('span').get_text()
    if result_temp[0] == '+':
        sector_change = float(result_temp[1:-1])
    else:
        sector_change = float(result_temp[:-1])
    sector_market_cap = results[1].find('span').get_text()
    sector_market_weight = float(results[2].find('span').get_text()[:-1])
    return sector_change,sector_market_cap,sector_market_weight

In [4]:
#Test get_sector_change_and_market_cap()
link = "https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25"
get_sector_change_and_market_cap(link)
#Should return (-0.40, $3.58T, 6.80)
#Note that neither the -0.40, nor the 6.80, end with a %sign

(0.96, '$8.95T', 12.85)

In [5]:
#Test get_us_sector_performance()
get_us_sector_performance()
#Should return (example: obviously the results will vary over time!)
#"""
#[('Telecommunication Services',
#  0.21,
#  '$1.74T',
#  2.0,
# 'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
# ('Materials',
#  0.22,
#  '$1.95T',
#  2.49,
#  'https://eresearch.fidelity.com/eresearch/marksectors/sectors/sectors_in_market.jhtml?tab=learn&sector=15'),
# ('Real Estate',ets_
#  -0.45,
#  '$1.22T',
#  2.7,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=60'),
# ('Utilities',
#  -0.33,
#  '$1.25T',
#  2.86,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=55'),
# ('Energy',
#  0.76,
#  '$3.90T',
#  5.83,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=10'),
# ('Consumer Staples',
#  -0.32,
#  '$3.58T',
#  6.8,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=30'),
# ('Industrials',
#  0.72,
#  '$4.31T',
#  9.83,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
# ('Consumer Discretionary',
#  1.03,
#  '$5.76T',
#  12.9,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
# ('Financials',
#  0.39,
#  '$7.45T',
#  13.71,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
# ('Health Care',
#  0.76,
#  '$5.70T',
#  14.71,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
# ('Information Technology',
#  0.81,
#  '$9.84T',
#  26.17,
#  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45')]
  
#"""

[('Information Technology',
  1.55,
  '$14.16T',
  27.76,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=45'),
 ('Health Care',
  0.36,
  '$7.61T',
  13.77,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=35'),
 ('Consumer Discretionary',
  0.96,
  '$8.95T',
  12.85,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=25'),
 ('Communication Services',
  0.6,
  '$6.07T',
  10.71,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=50'),
 ('Financials',
  2.28,
  '$7.31T',
  10.34,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=40'),
 ('Industrials',
  1.1,
  '$5.10T',
  8.11,
  'https://eresearch.fidelity.com/eresearch/markets_sectors/sectors/sectors_in_market.jhtml?tab=learn&sector=20'),
 ('C

<h3>Sorting</h3>

In [6]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort() #Sorts by the first element of the tuple
x

[('a', 23.2, 'b'), ('c', 17.4, 'f'), ('d', 29.2, 'z'), ('e', 1.74, 'bb')]

In [7]:
x = [('a',23.2,'b'),('c',17.4,'f'),('d',29.2,'z'),('e',1.74,'bb')]
x.sort(key=lambda k: k[1]) #Sorts by the element at position 1
x

[('e', 1.74, 'bb'), ('c', 17.4, 'f'), ('a', 23.2, 'b'), ('d', 29.2, 'z')]